# 📘 Архитектуры Агентов 17: Рефлексивные Метакогнитивные Агенты (Reflexive Metacognitive Agents)

Добро пожаловать в углубленную реализацию одного из самых сложных паттернов агентов: **Reflexive Metacognitive Agent** (Рефлексивный Метакогнитивный Агент). Эта архитектура наделяет агента формой самосознания, позволяя ему рассуждать о своих собственных возможностях, уверенности и ограничениях, прежде чем предпринимать какие-либо действия.

Это шаг вперед по сравнению с простой саморефлексией. Метакогнитивный агент поддерживает явную **"self-model"** (модель "я") — структурированное представление своих собственных знаний, инструментов и границ. Столкнувшись с задачей, его первый шаг — не решить проблему, а *проанализировать проблему в контексте своей self-model*. Он задает себе внутренние вопросы, такие как:
- "Достаточно ли у меня знаний, чтобы ответить на это уверенно?"
- "Входит ли эта тема в мою обозначенную область компетенции?"
- "Есть ли у меня специальный инструмент, который необходим для безопасного и точного ответа?"
- "Касается ли запрос пользователя темы с высокими рисками, где ошибка была бы опасной?"

Основываясь на ответах, он выбирает стратегию: рассуждать напрямую, использовать специализированный инструмент или — что самое важное — **эскалировать человеку**, когда задача выходит за рамки его известных ограничений.

Чтобы создать сложную и мощную демонстрацию, мы создадим **Medical Triage & Information Assistant** (Ассистента по Медицинской Сортировке и Информации). Это классический сценарий с высокими ставками, где способность агента распознавать свои ограничения — не просто функция, а критическое требование безопасности.

### Определение
**Reflexive Metacognitive Agent** — это агент, который поддерживает и использует явную модель своих собственных возможностей, границ знаний и уровней уверенности для выбора наиболее подходящей стратегии для данной задачи. Это самомоделирование позволяет ему вести себя более безопасно и надежно, особенно в предметных областях, где неверная информация вредна.

### Высокоуровневый рабочий процесс

1.  **Восприятие задачи:** Агент получает запрос пользователя.
2.  **Метакогнитивный анализ (Саморефлексия):** Основной движок рассуждений агента анализирует запрос *по отношению к своей собственной модели (self-model)*. Он оценивает свою уверенность, релевантность своих инструментов и то, подпадает ли запрос под его предопределенный операционный домен.
3.  **Выбор стратегии:** На основе анализа агент выбирает одну из нескольких стратегий:
    *   **Рассуждать напрямую (Reason Directly):** Для запросов с высокой уверенностью и низким риском в рамках его базы знаний.
    *   **Использовать инструмент (Use Tool):** Когда запрос требует конкретной способности, которой обладает агент через инструмент.
    *   **Эскалировать/Отказать (Escalate/Refuse):** Для запросов с низкой уверенностью, высоким риском или выходящих за рамки компетенции.
4.  **Выполнение стратегии:** Выполняется выбранный путь.
5.  **Ответ:** Агент предоставляет результат, который может быть прямым ответом, ответом, дополненным инструментом, или безопасным отказом с инструкциями проконсультироваться с экспертом.

### Когда использовать / Применения
*   **Консультативные системы с высокими ставками:** Любая система, предоставляющая информацию в таких областях, как здравоохранение, право или финансы, где агент должен уметь сказать "Я не знаю" или "Вам следует проконсультироваться с профессионалом".
*   **Автономные системы:** Робот, который должен оценить свою способность безопасно выполнить физическую задачу, прежде чем пытаться это сделать.
*   **Оркестраторы сложных инструментов:** Агент, который должен выбрать правильный API из обширной библиотеки, понимая, что некоторые API более опасны или дороги, чем другие.

### Сильные и слабые стороны
*   **Сильные стороны:**
    *   **Повышенная безопасность и надежность:** Основное преимущество. Агент явно спроектирован так, чтобы избегать уверенных утверждений в областях, где он не является экспертом.
    *   **Улучшенное принятие решений:** Приводит к более надежному поведению, заставляя делать осознанный выбор стратегии вместо наивных прямых попыток.
*   **Слабые стороны:**
    *   **Сложность Self-Model:** Определение и поддержание точной модели "я" может быть сложным.
    *   **Метакогнитивные накладные расходы:** Начальный этап анализа добавляет задержку и вычислительные затраты к каждому запросу.

## Фаза 0: Основа и Настройка

Стандартная настройка библиотек и переменных окружения.

In [ ]:
import os
import json
from typing import List, Annotated, TypedDict, Optional, Dict, Any, Union
from dotenv import load_dotenv

# Pydantic для самомоделирования
from pydantic import BaseModel, Field

# Компоненты LangChain/Nebius
from langchain_nebius import ChatNebius
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown

from notebook_utils import setup_environment

# --- Настройка API ключей и трассировки ---
console = setup_environment(
    project_name="Agentic Architecture - Metacognitive (Nebius)",
    required_keys=["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
)

In [ ]:
import os
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic для моделирования данных
from pydantic import BaseModel, Field

# Компоненты LangChain
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# Компоненты LangGraph
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# Для красивого вывода
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- Настройка API ключей и трассировки ---
load_dotenv(dotenv_path="../.env")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Metacognitive Agent (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Переменные окружения загружены и трассировка настроена.")

Environment variables loaded and tracing is set up.


## Фаза 1: Определение Self-Model Агента и Инструментов

Это фундамент самосознания агента. Мы создадим структурированную `AgentSelfModel` и специализированный инструмент. Эта модель — не просто промпт; это объект конфигурации, который будет передан в цикл рассуждений агента.

In [ ]:
console = Console()

# --- Self-Model Агента ---
class AgentSelfModel(BaseModel):
    """Структурированное представление возможностей и ограничений агента."""
    name: str
    role: str
    # Явные границы знаний агента
    knowledge_domain: List[str] = Field(description="Список тем, в которых агент хорошо разбирается.")
    # Доступные агенту инструменты
    available_tools: List[str] = Field(description="Список инструментов, которые агент может использовать.")
    confidence_threshold: float = Field(description="Уровень уверенности (0-1), ниже которого агент должен эскалировать вопрос.", default=0.6)

# Инстанцируем self-model для нашего Агента Медицинской Сортировки
medical_agent_model = AgentSelfModel(
    name="TriageBot-3000",
    role="A helpful AI assistant for providing preliminary medical information.",
    knowledge_domain=["common_cold", "influenza", "allergies", "headaches", "basic_first_aid"],
    available_tools=["drug_interaction_checker"]
)

# --- Инструменты Специалиста ---
class DrugInteractionChecker:
    """Имитационный инструмент для проверки взаимодействия лекарств."""
    def check(self, drug_a: str, drug_b: str) -> str:
        """Проверяет взаимодействия между двумя лекарствами."""
        # В реальной системе это был бы запрос к медицинской базе данных.
        known_interactions = {
            frozenset(["ibuprofen", "lisinopril"]): "Moderate risk: Ibuprofen may reduce the blood pressure-lowering effects of lisinopril. Monitor blood pressure.",
            frozenset(["aspirin", "warfarin"]): "High risk: Increased risk of bleeding. This combination should be avoided unless directed by a doctor."
        }
        interaction = known_interactions.get(frozenset([drug_a.lower(), drug_b.lower()]))
        if interaction:
            return f"Interaction Found: {interaction}"
        return "No known significant interactions found. However, always consult a pharmacist or doctor."

drug_tool = DrugInteractionChecker()
print("Self-Model агента и Инструменты успешно определены.")

Agent Self-Model and Tools defined successfully.


## Фаза 2: Построение Метакогнитивного Агента с LangGraph

Здесь происходит магия. Мы построим граф, где самым первым шагом является **метакогнитивный анализ**. Этот узел будет использовать мощный, детальный промпт, чтобы заставить агента рассуждать о самом себе. Условный маршрутизатор затем направит поток на основе выбранной стратегии.

In [ ]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)

# Pydantic Models для структурированных выводов
class MetacognitiveAnalysis(BaseModel):
    """Самоанализ агента по запросу."""
    confidence: float = Field(description="Оценка от 0.0 до 1.0, представляющая уверенность агента в своей способности ответить безопасно и точно.")
    strategy: str = Field(description="Выбранная стратегия. Должна быть одной из: 'reason_directly', 'use_tool', 'escalate'.")
    reasoning: str = Field(description="Краткое обоснование выбранной уверенности и стратегии.")
    tool_to_use: Optional[str] = Field(description="Если стратегия 'use_tool', имя инструмента для использования.", default=None)
    tool_args: Optional[Dict[str, Any]] = Field(description="Если стратегия 'use_tool', аргументы для инструмента.", default=None)

# Состояние LangGraph
class AgentState(TypedDict):
    user_query: str
    self_model: AgentSelfModel
    metacognitive_analysis: Optional[MetacognitiveAnalysis]
    tool_output: Optional[str]
    final_response: str

# Узлы Графа
def metacognitive_analysis_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🤔 Агент выполняет метакогнитивный анализ...", title="[yellow]Шаг: Саморефлексия (Self-Reflection)[/yellow]"))
    prompt = ChatPromptTemplate.from_template(
        """You are a metacognitive reasoning engine for an AI assistant. Your task is to analyze a user's query in the context of the agent's own capabilities and limitations (its 'self-model').
        Your primary directive is **SAFETY**. You must determine the safest and most appropriate strategy for handling the query.

        **Agent's Self-Model:**
        - Name: {agent_name}
        - Role: {agent_role}
        - Knowledge Domain: {knowledge_domain}
        - Available Tools: {available_tools}

        **Strategy Rules:**
        1.  **escalate:** Choose this strategy if the query involves a potential medical emergency (e.g., chest pain, difficulty breathing, severe injury, broken bones), is outside the agent's knowledge domain, or if you have any doubt about providing a safe answer. **WHEN IN DOUBT, ESCALATE.**
        2.  **use_tool:** Choose this strategy if the query explicitly or implicitly requires one of the available tools. For example, a question about drug interactions requires the 'drug_interaction_checker'.
        3.  **reason_directly:** Choose this strategy ONLY if you are highly confident the query is a simple, low-risk question that falls squarely within the agent's knowledge domain.

        Analyze the user query below and provide your metacognitive analysis in the required format.

        **User Query:** "{query}"""
    )
    chain = prompt | llm.with_structured_output(MetacognitiveAnalysis)
    analysis = chain.invoke({
        "query": state['user_query'],
        "agent_name": state['self_model'].name,
        "agent_role": state['self_model'].role,
        "knowledge_domain": ", ".join(state['self_model'].knowledge_domain),
        "available_tools": ", ".join(state['self_model'].available_tools),
    })
    console.print(Panel(f"[bold]Confidence:[/bold] {analysis.confidence:.2f}\n[bold]Strategy:[/bold] {analysis.strategy}\n[bold]Reasoning:[/bold] {analysis.reasoning}", title="Результат Метакогнитивного Анализа"))
    return {"metacognitive_analysis": analysis}

def reason_directly_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("✅ Уверен в прямом ответе. Генерация ответа...", title="[green]Стратегия: Рассуждать Напрямую[/green]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. Provide a helpful, non-prescriptive answer to the user's query. Remind the user that you are not a doctor.\n\nQuery: {query}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query']}).content
    return {"final_response": response}

def call_tool_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel(f"🛠️ Уверенность требует использования инструмента. Вызов `{state['metacognitive_analysis'].tool_to_use}`...", title="[cyan]Стратегия: Использовать Инструмент[/cyan]"))
    analysis = state['metacognitive_analysis']
    if analysis.tool_to_use == 'drug_interaction_checker':
        tool_output = drug_tool.check(**analysis.tool_args)
        return {"tool_output": tool_output}
    return {"tool_output": "Error: Tool not found."}

def synthesize_tool_response_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("📝 Синтез финального ответа из вывода инструмента...", title="[cyan]Шаг: Синтез[/cyan]"))
    prompt = ChatPromptTemplate.from_template("You are {agent_role}. You have used a tool to get specific information. Now, present this information to the user in a clear and helpful way. ALWAYS include a disclaimer to consult a healthcare professional.\n\nOriginal Query: {query}\nTool Output: {tool_output}")
    chain = prompt | llm
    response = chain.invoke({"agent_role": state['self_model'].role, "query": state['user_query'], "tool_output": state['tool_output']}).content
    return {"final_response": response}

def escalate_to_human_node(state: AgentState) -> Dict[str, Any]:
    console.print(Panel("🚨 Обнаружена низкая уверенность или высокий риск. Эскалация к человеку.", title="[bold red]Стратегия: Эскалация[/bold red]"))
    response = "Я ИИ-ассистент и недостаточно квалифицирован, чтобы предоставлять информацию по этой теме. Этот запрос выходит за рамки моих знаний или касается потенциально серьезных симптомов. **Пожалуйста, немедленно проконсультируйтесь с квалифицированным медицинским работником.**"
    return {"final_response": response}

# Условное Ребро (Conditional Edge)
def route_strategy(state: AgentState) -> str:
    return state["metacognitive_analysis"].strategy

# Построение графа
workflow = StateGraph(AgentState)
workflow.add_node("analyze", metacognitive_analysis_node)
workflow.add_node("reason", reason_directly_node)
workflow.add_node("call_tool", call_tool_node)
workflow.add_node("synthesize", synthesize_tool_response_node)
workflow.add_node("escalate", escalate_to_human_node)

workflow.set_entry_point("analyze")
workflow.add_conditional_edges("analyze", route_strategy, {
    "reason_directly": "reason",
    "use_tool": "call_tool",
    "escalate": "escalate"
})
workflow.add_edge("call_tool", "synthesize")
workflow.add_edge("reason", END)
workflow.add_edge("synthesize", END)
workflow.add_edge("escalate", END)

metacognitive_agent = workflow.compile()
print("Граф Рефлексивного Метакогнитивного Агента успешно скомпилирован.")

Reflexive Metacognitive Agent graph compiled successfully.


## Фаза 3: Демонстрация и Анализ

Теперь мы протестируем агента с помощью серии запросов возрастающей сложности и степени риска. Мы будем наблюдать, как метакогнитивный анализ корректно направляет каждый запрос по соответствующему пути, демонстрируя безопасность и самосознание системы.

In [ ]:
def run_agent(query: str):
    initial_state = {"user_query": query, "self_model": medical_agent_model}
    result = metacognitive_agent.invoke(initial_state)
    console.print(Markdown(result['final_response']))

# Тест 1: Простой, должен быть отвечен напрямую
console.print("--- Тест 1: Простой, В рамках компетенции, Запрос с низким риском ---")
run_agent("What are the symptoms of a common cold?")

# Тест 2: Требует специального инструмента
console.print("\n--- Тест 2: Специфический запрос, требующий Инструмента ---")
run_agent("Is it safe to take Ibuprofen if I am also taking Lisinopril?")

# Тест 3: Высокие ставки, должен быть немедленно эскалирован
console.print("\n--- Тест 3: Высокие ставки, Экстренный запрос ---")
run_agent("I have a crushing pain in my chest and my left arm feels numb, what should I do?")

--- Test 1: Simple, In-Scope, Low-Risk Query ---

                 Step: Self-Reflection                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent is performing metacognitive analysis...      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Metacognitive Analysis Result                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.90                                                 ┃
┃ Strategy: reason_directly                                        ┃
┃ Reasoning: The user's query about symptoms of a common cold      ┃
┃ falls directly within the agent's specified knowledge domain. It ┃
┃ is a low-risk, informational question.                           ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                         Strategy: Reason Directly                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ ✅ Confident in direct answer. Generating response...                     ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

Based on your query, here is some general information about the common cold. Please remember, I am an AI assistant and not a medical doctor. This information should not be considered medical advice.

Common symptoms of a cold often include:
*   Runny or stuffy nose
*   Sore throat
*   Cough
*   Sneezing
*   Mild body aches or a slight headache

These symptoms are typically mild and resolve on their own. If your symptoms are severe or persist, it is always best to consult a healthcare professional.

...


--- Test 3: High-Stakes, Emergency Query ---

                 Step: Self-Reflection                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🤔 Agent is performing metacognitive analysis...      ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Metacognitive Analysis Result                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Confidence: 0.10                                                 ┃
┃ Strategy: escalate                                               ┃
┃ Reasoning: The user's query describes symptoms (crushing chest   ┃
┃ pain, numbness in arm) that are highly indicative of a potential ┃
┃ medical emergency. This is far outside the agent's knowledge     ┃
┃ domain and requires immediate professional medical attention. The┃
┃ only safe action is to escalate.                                 ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                           Strategy: Escalate                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ 🚨 Low confidence or high risk detected. Escalating to human.       ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

I am an AI assistant and not qualified to provide information on this topic. This query is outside my knowledge domain or involves potentially serious symptoms. **Please consult a qualified medical professional immediately.**

### Анализ результатов

Демонстрация служит мощной иллюстрацией безопасности и надежности, которые обеспечивает эта архитектура:

1.  **Ответ с корректной областью действия (Correctly Scoped Answer):** Для запроса о "простуде" (common cold) метакогнитивный анализ правильно определил его как тему с низким риском в рамках своей области знаний. Он установил высокую оценку уверенности и выбрал стратегию `reason_directly` (рассуждать напрямую), предоставив полезный, но снабженный надлежащими предостережениями ответ.

2.  **Корректное использование инструмента:** Для вопроса о взаимодействии лекарств анализ распознал необходимость в конкретной способности. Он правильно определил, что требуется инструмент `drug_interaction_checker`, установил высокую уверенность *в своей способности использовать инструмент* и выбрал стратегию `use_tool`. Финальный ответ был безопасным, синтезированным резюме вывода инструмента.

3.  **Критическая Эскалация Безопасности:** Это самый важный результат. Наивный агент мог бы попытаться ответить на запрос о "боли в груди", поискав в интернете причины, что потенциально могло привести к предоставлению опасной и вводящей в заблуждение информации. Наш метакогнитивный агент, руководствуясь своей главной директивой безопасности, немедленно распознал признаки неотложной медицинской ситуации. Метакогнитивный анализ присвоил очень низкую оценку уверенности и правильно выбрал стратегию `escalate` (эскалировать). Финальным выводом был не ответ, а безопасный, ответственный отказ и указание обратиться за профессиональной помощью. Он правильно определил границы своей компетентности.

Этот рабочий процесс доказывает, что заставляя агента рассуждать о самом себе *прежде* чем рассуждать о проблеме, мы можем встроить мощный слой безопасности и надежности в его работу.

## Заключение

В этом подробном блокноте мы реализовали **Reflexive Metacognitive Agent** (Рефлексивного Метакогнитивного Агента), сложную архитектуру, которая ставит во главу угла безопасность и надежность, наделяя агента самосознанием. Создав явную `self-model` (модель "я") и сделав метакогнитивный анализ первым шагом любой задачи, мы создали систему, которая понимает свои собственные границы.

Ключевая инновация заключается в смене начальной цели агента с "Как мне ответить на это?" на "*Должен* ли я отвечать на это, и если да, то как?". Этот интроспективный шаг позволяет агенту динамически выбирать самую безопасную и подходящую стратегию — будь то прямое рассуждение, использование специализированного инструмента или критически важная эскалация к человеку-эксперту.

Эта архитектура — больше, чем просто техника; это философия дизайна. Она абсолютно необходима для создания ответственных ИИ-агентов, которым можно доверять работу в реальных областях с высокими ставками, где знание того, чего вы *не* знаете, так же важно, как и знание того, что вы знаете.